In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import os
from BNS_JT import trans
from BNS_JT import variable, cpm
import numpy as np

import BNS_JT
from BNS_JT import gen_bnb
import copy

HOME = os.getcwd()


In [2]:
import importlib
importlib.reload(gen_bnb)

<module 'BNS_JT.gen_bnb' from 'c:\\Users\\jb622s\\git\\BNS-JT\\BNS_JT\\gen_bnb.py'>

# Results of general B&B algorithm to reliability analysis

Complete branch and bound, deterministic inference, and indepdent components

# Problem
## Network

In [3]:
# Network
node_coords = {'n1': (-2, 3),
               'n2': (-2, -3),
               'n3': (2, -2),
               'n4': (1, 1),
               'n5': (0, 0)}

arcs = {'e1': ['n1', 'n2'],
	'e2': ['n1', 'n5'],
	'e3': ['n2', 'n5'],
	'e4': ['n3', 'n4'],
	'e5': ['n3', 'n5'],
	'e6': ['n4', 'n5']}

arcs_avg_kmh = {'e1': 40,
                'e2': 40,
                'e3': 40,
                'e4': 30,
                'e5': 30,
                'e6': 20}

arc_lens_km = trans.get_arcs_length(arcs, node_coords)
arc_times_h = {k: v/arcs_avg_kmh[k] for k, v in arc_lens_km.items()}

comps_name = [k for k in arcs] # *TODO* this is not necessary if branch's down and up are defined by dictionary (instead of list)


od_pair = ('n1', 'n3')


# Component events
no_arc_st = 3 # number of component states 
delay_rat = [10, 2, 1] # delay in travel time given each component state (ratio)
varis = {}
for k, v in arcs.items():
    varis[k] = variable.Variable( name=k, B = np.eye( no_arc_st ), values = [arc_times_h[k]*np.float64(x) for x in delay_rat])


# plot graph
G = nx.Graph()
for k, x in arcs.items():
    G.add_edge(x[0], x[1], time=arc_times_h[k], label=k)

for k, v in node_coords.items():
    G.add_node(k, pos=v)

pos = nx.get_node_attributes(G, 'pos')
edge_labels = nx.get_edge_attributes(G, 'label')

fig = plt.figure()
ax = fig.add_subplot()
nx.draw(G, pos, with_labels=True, ax=ax)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
fig.savefig(os.path.join(HOME, 'graph.png'), dpi=200)

## Events: Independent component events

In [4]:
cpms = {}

# Component events
for k in arcs:
    cpms[k] = cpm.Cpm( variables=[ varis[k] ], no_child = 1, C = np.array([1,2,3])-1, p = [0.1, 0.2, 0.7] ) 

# Damage observation
C_o = np.array([[1,1], [2,1], [3,1], [1,2], [2,2], [3,2], [1,3], [2,3], [3,3]])-1
p_o = np.array([0.95, 0.04, 0.01, 0.3, 0.5, 0.2, 0.01, 0.19, 0.8]).T
for (i,k) in enumerate(arcs):
    name = 'o' + str(i+1)

    varis[name] = variable.Variable( name=name, B = np.eye( no_arc_st ), values = [0,1,2] ) # observation that e_i = 0, 1, or 2 ** TO DISCUSS: probably values in dictionary..?
    cpms[name] = cpm.Cpm( variables=[ varis[name], varis[k] ], no_child = 1, C = C_o, p = p_o )

print(cpms['e1'])
print(cpms['o1'])

Cpm(variables=['e1'], no_child=1, C=[[0]
 [1]
 [2]], p=[[0.1]
 [0.2]
 [0.7]]
Cpm(variables=['o1', 'e1'], no_child=1, C=[[0 0]
 [1 0]
 [2 0]
 [0 1]
 [1 1]
 [2 1]
 [0 2]
 [1 2]
 [2 2]], p=[[0.95]
 [0.04]
 [0.01]
 [0.3 ]
 [0.5 ]
 [0.2 ]
 [0.01]
 [0.19]
 [0.8 ]]


# System event quantification by general B&B algorithm

## System function

In [5]:
def get_time_and_path( comps_st, od_pair, arcs, vari ):
    G = nx.Graph()
    for k, x in arcs.items():
        c_st = comps_st[k]
        G.add_edge(x[0], x[1], time=vari[k].values[c_st])

    path = nx.shortest_path( G, source = od_pair[0], target = od_pair[1], weight = 'time' )
    time = nx.shortest_path_length( G, source = od_pair[0], target = od_pair[1], weight = 'time' )

    return time, path


def sf_min_path( comps_st, od_pair, arcs, vari, thres, sys_val_itc ):

    sys_val, path = get_time_and_path( comps_st, od_pair, arcs, vari )

    if sys_val > thres*sys_val_itc:
        sys_st = 'fail'
    else:
        sys_st = 'surv'

    if sys_st == 'surv': # in this case we know how to find out minimally required component state
        min_comps_st = {}
        for i in range(len(path)-1):
            nodes_i = [path[i], path[i+1]]
            nodes_i_rev = [path[i+1], path[i]] # reversed pair (arcs are bi-directional)
            arc_i = next((k for k, v in arcs.items() if v == nodes_i or v == nodes_i_rev), None)
            min_comps_st[arc_i] = comps_st[arc_i]
    
    else: # sys_st == 'fail'
        min_comps_st = None

    return sys_val, sys_st, min_comps_st

# Intact state of component vector
comps_st_itc = {k: len(varis[k].B[0])-1 for k in arcs} # intact state (i.e. the highest state)
sys_val_itc, path_itc = get_time_and_path( comps_st_itc, od_pair, arcs, varis )

# defines the system failure event
thres = 2 

# Given a system function, i.e. sf_min_path, it should be represented by a function that only has "comps_st" as input.
sys_fun = lambda comps_st : sf_min_path( comps_st, od_pair, arcs, varis, thres, sys_val_itc ) # TODO: branch needs to have states defined in dictionary instead of list.

# Branch and Bound: Incomplete results, producing unknown branches

In [6]:
import importlib
importlib.reload(gen_bnb)

<module 'BNS_JT.gen_bnb' from 'c:\\Users\\jb622s\\git\\BNS-JT\\BNS_JT\\gen_bnb.py'>

In [7]:
varis_comp = {}
for k in arcs:
    varis_comp[k] = copy.deepcopy( varis[k] )
    

no_sf, rules, rules_st, brs, sys_res = gen_bnb.do_gen_bnb( sys_fun, varis_comp, max_br=5 ) # Complete 

[Iteration 1]..
The # of found non-dominated rules: 0
The # of branches: 0
---
[Iteration 2]..
The # of found non-dominated rules: 1
The # of branches: 1
---
[Iteration 3]..
The # of found non-dominated rules: 2
The # of branches: 1
---
[Iteration 4]..
The # of found non-dominated rules: 3
The # of branches: 1
---
[Iteration 5]..
The # of found non-dominated rules: 3
The # of branches: 1
---
[Iteration 6]..
The # of found non-dominated rules: 4
The # of branches: 1
---
[Iteration 7]..
The # of found non-dominated rules: 4
The # of branches: 1
---
[Iteration 8]..
The # of found non-dominated rules: 5
The # of branches: 1
---
[Iteration 9]..
The # of found non-dominated rules: 5
The # of branches: 2
---
[Iteration 10]..
The # of found non-dominated rules: 6
The # of branches: 2
---
[Iteration 11]..
The # of found non-dominated rules: 6
The # of branches: 4
---
[Iteration 12]..
The # of found non-dominated rules: 7
The # of branches: 4
---
[Iteration 13]..
The # of found non-dominated rul

# MCS on unknown branches

%% 141223